# Import required libraries

In [5]:
import pandas as pd
import sklearn.model_selection as skms
import sklearn.metrics as skmt
import matplotlib.pyplot as plt
import numpy as np

# Utilities functions

Function *compute_confusion_matrix* compute and show the confusion matrix

In [6]:
def compute_confusion_matrix(sufix,y_test, y_pred):

    labels = [False, True]

    # Compute the confusion matrix
    conf_matrix = skmt.confusion_matrix(y_test, y_pred, labels=labels)
    print(f"[{sufix}]confusion matrix:\n",conf_matrix)

    if len(conf_matrix[0]) == 1:
        print("Skipping ConfusionMatrixDisplay because a single label was found in 'y_true' and 'y_pred'.")
        return

    # Create a display object for the confusion matrix
    disp = skmt.ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["No Granted", "Granted"])

    # Plot the confusion matrix
    disp.plot(cmap=plt.cm.Blues, values_format=".0f")
    plt.title(f"[{sufix}] Confusion Matrix")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

Function *evaluate_classifier* return Accuracy and Balanced Accuracy scores, and the confusion matrix

In [7]:
def evaluate_classifier(y_test, y_pred):
    # Compute and print Accuracy
    accuracy_score = skmt.accuracy_score(y_test, y_pred)
    print('Accuracy: %.2f' % (accuracy_score*100))

    # Compute and print Balanced Accuracy
    balanced_accuracy = skmt.balanced_accuracy_score(y_test, y_pred)
    print('Balanced Accuracy: %.2f' % (balanced_accuracy*100))

    # Compute and show confusion matrix
    compute_confusion_matrix('General',y_test, y_pred)

## Load dataset

In [8]:
dataset_ulr = 'https://raw.githubusercontent.com/leticiaarco/practical_ethical_ai_resources/refs/heads/main/scholarship.csv'
data = pd.read_csv(dataset_ulr)
data.head()
df = pd.DataFrame(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   salary       120 non-null    float64
 1   gender       120 non-null    object 
 2   married      120 non-null    object 
 3   achievement  120 non-null    float64
 4   scholarship  120 non-null    bool   
dtypes: bool(1), float64(2), object(2)
memory usage: 4.0+ KB


## Handle data

Transform feature values

In [9]:
transformed_data = data

transformed_data['gender'] = transformed_data['gender'].map({
    'F':1,
    'M':0,
})

transformed_data['married'] = transformed_data['married'].map({
    'Y':1,
    'N':0,
})

Split dataset by predictive features (X) and objective feature (y)

In [10]:
X = transformed_data.drop('scholarship', axis=1)
y = transformed_data['scholarship']

Split dataset in train and test sets

In [11]:
X_train, X_test, y_train, y_test = skms.train_test_split(X, y, test_size=0.3, random_state=0)

# Train the classifier

Import

In [18]:
import tensorflow.keras.models as tkm
import tensorflow.keras.layers as tkl

Training occurs over epochs, and each epoch is split into batches
<br>
**Epoch**: One pass through all of the rows in the training dataset
<br>
**Batch**: One or more samples considered by the model within an epoch before weights are updated

**Original setting:** **12 neurons** in the first and **8 neurons** in the second hiddden layers, **150 number of epochs**, and **batch size 10**

In [19]:
# Initialize keras sequential model
model = tkm.Sequential()
model.add(tkl.Input(shape=(4,)))
model.add(tkl.Dense(12, activation='relu'))
model.add(tkl.Dense(8, activation='relu'))
model.add(tkl.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model on the trainset (epochs=150, batch_size=10 )
model.fit(X_train, y_train, epochs=150, batch_size=10)

# Evaluate keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0834 - loss: 879.2312 
Epoch 2/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9577 - loss: 10.3906     
Epoch 3/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9378 - loss: 25.5693     
Epoch 4/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9280 - loss: 35.1287     
Epoch 5/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9359 - loss: 30.5231     
Epoch 6/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9152 - loss: 40.0728 
Epoch 7/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9153 - loss: 37.8655 
Epoch 8/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9116 - loss: 32.4837 
Epoch 9/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9423 - loss: 18.3353    
Epoch 10/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9156 - loss: 22.4787 
Epoch 11/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9566 - loss: 9.0427      
Epoch 12/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0

### Play with the parameter settings to improve results

**Increase the number of epochs:** number of epochs = 1000

In [20]:
# Fit the model on the trainset (epochs=1000, batch_size=10 )
model.fit(X_train, y_train, epochs=1000, batch_size=10)

# Evaluate keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9210 - loss: 6.2151  
Epoch 2/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8290 - loss: 1.4732 
Epoch 3/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9481 - loss: 3.8606     
Epoch 4/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9487 - loss: 2.5805     
Epoch 5/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7918 - loss: 3.1957 
Epoch 6/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9202 - loss: 7.3488     
Epoch 7/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9425 - loss: 2.9863     
Epoch 8/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7150 - loss: 1.2996     
Epoch 9/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9502 - loss: 1.9893     
Epoch 10/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7199 - loss: 1.4274     
Epoch 11/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9348 - loss: 4.4892     
Epoch 12/1000
9/9 ━━━━━━━━━━━━

**Increase batch size:** batch size = 80

In [21]:
# Fit the model on the trainset (epochs=150, batch_size=80)
model.fit(X_train, y_train, epochs=150, batch_size=80)

# Evaluate keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9274 - loss: 0.9900  
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9274 - loss: 1.5630 
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9274 - loss: 1.7519 
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9274 - loss: 1.6292 
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9315 - loss: 1.1265 
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9077 - loss: 0.3040 
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9395 - loss: 0.2091 
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9274 - loss: 1.4198 
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9274 - loss: 2.1488 
Epoch 10/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9315 - loss: 2.3578 
Epoch 11/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9274 - loss: 2.0682 
Epoch 12/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9274 - l

**Increase the number of neurons per hidden layers:**  24 neurons in the first and 12 neurons in the second hiddden layers

In [22]:
# Initialize keras sequential model
model = tkm.Sequential()
model.add(tkl.Input(shape=(4,)))
model.add(tkl.Dense(24, activation='relu'))
model.add(tkl.Dense(12, activation='relu'))
model.add(tkl.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model on the trainset (epochs=150, batch_size=10 )
model.fit(X_train, y_train, epochs=150, batch_size=10)

# Evaluate keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0639 - loss: 4827.8340    
Epoch 2/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0656 - loss: 1486.8048     
Epoch 3/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9247 - loss: 19.0135    
Epoch 4/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9063 - loss: 66.8687 
Epoch 5/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8933 - loss: 96.7363  
Epoch 6/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9277 - loss: 70.4287    
Epoch 7/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9444 - loss: 54.2285    
Epoch 8/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8983 - loss: 89.6008  
Epoch 9/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9470 - loss: 44.5162    
Epoch 10/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9499 - loss: 39.8816    
Epoch 11/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9253 - loss: 54.2927 
Epoch 12/150
9/9 ━━━━━━━━━━━━━━━━━━

**Setting with all suggested modifications:** **24 neurons** in the first and **12 neurons** in the second hiddden layers, **1000 number of epochs**, and **batch size 80**

In [23]:
# Initialize keras sequential model
model = tkm.Sequential()
model.add(tkl.Input(shape=(4,)))
model.add(tkl.Dense(24, activation='relu'))
model.add(tkl.Dense(12, activation='relu'))
model.add(tkl.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model on the trainset (epochs=1000, batch_size=80 )
model.fit(X_train, y_train, epochs=1000, batch_size=80)

# Evaluate keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0685 - loss: 2737.2307
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0726 - loss: 2416.4124 
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0726 - loss: 2120.7769 
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0685 - loss: 1837.9548 
Epoch 5/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0726 - loss: 1519.1427 
Epoch 6/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0726 - loss: 1245.1224 
Epoch 7/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0726 - loss: 956.6553 
Epoch 8/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0601 - loss: 682.3863 
Epoch 9/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0685 - loss: 400.4117 
Epoch 10/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0685 - loss: 137.0284 
Epoch 11/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9274 - loss: 3.2718 
Epoch 12/1000
2/2 ━━━━━━━━━━━━━━━━━━━━